# CRUZAMENTO DE SETORES CENSITÁRIOS E BAIRROS

## Importação das bibliotecas

In [ ]:
import geopandas as gpd
import folium
from folium import GeoJsonTooltip
from shapely.geometry import Point

## Carregamento dos Dados
### Leitura dos dados com geopandas e conversão em dataframe

 Mapa dos Setores Censitários do IBGE - 2022

In [ ]:
setores_censitarios = '/Users/gabriel.marchesi/Desktop/Estatística/Python/pythonProject/SMPAE/Censo2022/SetoresxBairros/RS_Malha_Preliminar_2022/RS_Malha_Preliminar_2022.shp'
gdf_setores = gpd.read_file(setores_censitarios)

Mapa dos bairros - 2016

In [ ]:
bairros = '/Users/gabriel.marchesi/Desktop/Estatística/Python/pythonProject/SMPAE/Censo2022/SetoresxBairros/Bairros_2016/Bairros_2016.shp'
gdf_bairros = gpd.read_file(bairros)

## Tratamento dos Dados

No dataframe dos setores, filtrar somente o município de Porto Alegre e criar um novo dataframe

In [ ]:
gdf_setores_poa = gdf_setores[gdf_setores['CD_MUN'] == '4314902'].copy()

Converter os dataframes para o sistema de coordenadas desejado

Definir o sistema de coordenadas

In [ ]:
crs_desejado = """PROJCS["TM-POA",GEOGCS["SIRGAS 2000",DATUM["Sistema_de_Referencia_Geocentrico_para_las_AmericaS_2000",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6674"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-51],PARAMETER["scale_factor",0.999995],PARAMETER["false_easting",300000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]"""

Converter o dataframe de setores para o sistema de coordenadas desejado

In [ ]:
gdf_setores_poa_crs = gdf_setores_poa.to_crs(crs_desejado)

Converter o dataframe de bairros para o sistema de coordenadas desejado

In [ ]:
gdf_bairros_crs = gdf_bairros.to_crs(crs_desejado)

## Visualização Gráfica

Criar o mapa de Setores Censitários PoA

In [ ]:
fmap = folium.Map(location=[-30.0486439627364, -51.16280293048624], zoom_start=12)
feat_geojson_setores = folium.features.GeoJson(gdf_setores_poa_crs, style_function=lambda x: {'fillColor': '#cce6ff', 'color': 'blue', 'weight': 1})
fmap.add_child(feat_geojson_setores)
fmap.save('setores_poa.html')

Criar o mapa de Bairros PoA

In [ ]:
fmap = folium.Map(location=[-30.0486439627364, -51.16280293048624], zoom_start=12)
feat_geojson_bairros = folium.features.GeoJson(gdf_bairros_crs, style_function=lambda x: {'fillColor': '#cce6ff', 'color': 'black', 'weight': 2})
fmap.add_child(feat_geojson_bairros)
fmap.save('bairros_poa.html')

## Intersecção entre Setores Censitários e Bairros

Fazendo a interseção entre os dois dataframes

In [ ]:
gdf_intersecao = gpd.overlay(gdf_bairros_crs, gdf_setores_poa_crs, how='intersection')

Criar o mapa da interseção entre Setores Censitários e Bairros PoA

In [ ]:
fmap_intersecao = folium.Map(location=[-30.0486439627364, -51.16280293048624], zoom_start=12)

Adicionando as camadas com os estilos especificados

In [ ]:
feat_geojson_bairros = folium.features.GeoJson(gdf_intersecao,
                                               style_function=lambda x: {'fillColor': '#cce6ff', 'color': 'black', 'weight': 2},
                                               tooltip=GeoJsonTooltip(fields=['NOME', 'CD_SETOR'], aliases=['Bairro', 'Código do Setor'], labels=True, sticky=True))
feat_geojson_setores = folium.features.GeoJson(gdf_setores_poa_crs, style_function=lambda x: {'fillColor': '#cce6ff', 'color': 'blue', 'weight': 1})
fmap_intersecao.add_child(feat_geojson_bairros)
fmap_intersecao.add_child(feat_geojson_setores)
fmap_intersecao.save('intersecao_bairros_setores_poa.html')

## Cálculo dos Centróides dos Setores Censitários

In [ ]:
gdf_setores_poa_crs['centroide'] = gdf_setores_poa_crs['geometry'].centroid

Atribuir valores aos bairros

In [ ]:
for idx, row in gdf_setores_poa_crs.iterrows():
    point = row['centroide']
    nearest_bairro = gdf_bairros_crs.distance(point).idxmin()
    gdf_setores_poa_crs.at[idx, 'bairro'] = gdf_bairros_crs.loc[nearest_bairro, 'NOME']

gdf_setores_poa_crs

## Exportar para Excel

In [ ]:
gdf_setores_poa_crs.to_excel('bairros_setores_censitarios.xlsx', index=False)